In [1]:
# import statements
from Tools.toolbox import *
from Tools.functions import *
import csv
import pandas as pd

plt.rcParams['font.size'] = 15

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
dates_ebola = []
countries_ebola = []
with open('data/ebola_sorted.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        countries_ebola.append(row[0])
        dates_ebola.append(row[4])

data = {
    'Country': countries_ebola,
    'Date Onset Inferred': dates_ebola
}

df = pd.DataFrame(data)
df = df[df['Date Onset Inferred'] != 'NA']
countries = df.groupby('Country')
df_guinea = countries.get_group('Guinea')
dates_guinea = [date for date in df_guinea['Date Onset Inferred']]
df_liberia = countries.get_group('Liberia')
dates_liberia = [date for date in df_liberia['Date Onset Inferred']]
df_sierra = countries.get_group('Sierra Leone')
dates_sierra = [date for date in df_sierra['Date Onset Inferred']]
def sort_dates(dates):
    def date_key(date_string):
        return datetime.strptime(date_string, '%d/%m/%Y')
    return sorted(dates, key=date_key)
dates_all = sort_dates(df['Date Onset Inferred'])

In [3]:
date_nums_guinea = [convert_date_to_numberB(date, dates_guinea[0]) for date in dates_guinea]
date_nums_liberia = [convert_date_to_numberB(date, dates_liberia[0]) for date in dates_liberia]
date_nums_sierra = [convert_date_to_numberB(date, dates_sierra[0]) for date in dates_sierra]
date_nums_all = [convert_date_to_numberB(date, dates_all[0]) for date in dates_all]

In [4]:
observed_guinea = distribute(date_nums_guinea)
observed_liberia = distribute(date_nums_liberia)
observed_sierra = distribute(date_nums_sierra)
observed_all = distribute(date_nums_all)

In [5]:
event_times = observed_all

In [6]:
n = binary_search(event_times, 300)
initial_paras = [1, 1, 0.1]
bnds = ((0.01, 2), (0.01, 2), (1e-5, 1))
intensity = lambda t, paras: intensity_constructor(t, paras, event_times, ExpKernel, ConstBase, depth=30)
result = MLE_de(intensity, event_times[:n], model="constexp", bounds=bnds, maxiter=100)
optimized_params200 = result.x
print(optimized_params200)

[1.00465181 1.01259637 0.36621172]


In [10]:
n = binary_search(event_times, 300)
paras = optimized_params200
samples300B = []
for _ in range(100):
    samples300B.append(simulate_cluster_structure(ExpKernel(*paras[:2]), ConstBase(paras[-1]), event_times[:n], 360))

In [11]:
n = binary_search(event_times, 300)
paras = optimized_params200
intensity = lambda t, times: intensity_constructor(t, paras, times, ExpKernel, ConstBase, depth=30)
samples300T = []
for _ in range(1):
    samples300T.append(simulate_hawkes_process(intensity, event_times[:n], 360))

In [ ]:
n = binary_search(event_times, 300)
m = binary_search(event_times, 360)
for sample in samples300B:
    plt.plot(sample[n:], range(n, len(sample)), color="k", alpha=0.1)
plt.plot(event_times[:m], range(m), color='r')
plt.title('Simulation with Branching Algorithm')
plt.xlabel("Days")
plt.ylabel("N(t)")
plt.show()

In [ ]:
n = (360 - 300) // 7
n300 = binary_search(event_times, 300)
n360 = binary_search(event_times, 360)


samples300 = [sample[n300:] for sample in samples300B]
data300 = [[] for _ in range(n)]
for sample in samples300:
    for i in range(n):
        data300[i].append(count_by_days(sample, n)[i])
data_a300 = count_by_days(event_times[n300:n360], n)
plt.boxplot(data300)
plt.scatter(range(1, n+1, 1), data_a300, color="r")
plt.plot(range(1, n+1, 1), data_a300, color="r")
plt.xlabel("Prediction weeks")
plt.ylabel("Weekly frequency")
plt.show()